In [2]:
import torch
import numpy as np
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(1)
torch.cuda.current_device()

1

In [3]:
import pandas as pd

data = {
    'Text': [
        """I have never owned a pair of apple airpods before so I have nothing to base my purchase on but I purchased these on Amazon for ninety nine bucks and the apple site sells these for one hundred twentynine bucks
I checked to see if the serial number checks out on the apple support site it did identify as 2nd gen airpods but stated it could not validate my purchase date that doesnt prove much
The airpods seem to pass the sniff test on my bluetooth settings
My box states that it was assembled in Vietnam I checked online and it states that it should say assembled in China for some and other sources state that assembled in Vietnam is fine
When I connect the airpods to my iphone it does offer me 6 months of apple music if they were fake I dont think offer would be available to me
I do notice fast battery life the moment I take them off the charger and begin using them I notice them turning to 99 full in about 5 minutes without any use just wearing them in my ears the sound is not what I would expect from Apple some crackling noises but maybe I'm just nitpicking the sound on music is pretty good when Im cranking the volume all the way and listening to music people can hear it from 3 feet away in a quiet room when using bluetooth on calls Im told that my voice is coming crystal clear
Everything I checked seems to check out and these are real 2nd gen airpods but my skeptical mind tells me these are really good fakes
In any case these were a hundred bucks and work pretty good I will enjoy these""",
        
        """I have never owned un par of apple airpods before so I have nothing to base my compra on But I purchased these on Amazon for ninety nine bucks and the apple site sells these for one hundred twenty-nine bucks
I checked to see if the serial number checks out en the apple support site It did identify as 2nd gen airpods but stated it could not validate my fecha de compra That doesn't prove mucho
The airpods seem to pass the sniff test en my bluetooth settings
My box states that it was assembled en Vietnam I checked online and it states that it should say "Assembled en China" for some and other fuentes state that "Assembled en Vietnam" is fine
When I connect the airpods to my iphone it does offer me 6 months of apple música If they were fake I don't think oferta would be available to me
I do notice fast battery vida The moment I take them off the charger and begin using them I notice them turning to 99 full en about 5 minutes without any uso just wearing them en my ears The sound is not what I would expect from Apple some crackling ruidos pero maybe I'm just nitpicking The sound en música is pretty good When I'm cranking the volumen all the way and listening to música people can hear it from 3 feet away en a quiet room When using bluetooth en calls I'm told that my voz is coming crystal clear
Everything I checked seems to check out and these are real 2nd gen airpods but my skeptical mente tells me these son really good fakes
In any caso these were a hundred bucks and work pretty bien I will enjoy these"""
    ]
}

df = pd.DataFrame(data)

df.head()


,Text
0,I have never owned a pair of apple airpods bef...
1,I have never owned un par of apple airpods bef...


In [4]:
len(df.iloc[1]['Text'].split(' '))

287

In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(1)
torch.cuda.current_device()

1

In [6]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=1)
classifier_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=1)

In [11]:
import math


def classify_point_sentiment(point):
    label_candidate = ['positive', 'negative', 'neutral']
    results = classifier_pipeline(point, label_candidate)
    return results['labels'][0]

def classify_point_categories(point):
    label_candidate = ['Performance','Quality','Length','Durability','Functionality','Comfort','Customer Service','Material']
    results = classifier_pipeline(df['Text'][0], label_candidate)
    cat = results['labels'][0]
    sub_cat = results['labels'][1]
    return cat, sub_cat

def get_points(my_text):
    my_results = []
    words = my_text.split(' ')

    words_count = 50

    for i in range(math.ceil(len(words) / words_count)):
        print(i)
        my_string = " ".join(words[i*words_count:(i*words_count)+words_count])
        r = summarizer(my_string, max_length=28,
                       min_length=15, do_sample=False)
        
        cat, sub_cat =classify_point_categories(r[0]['summary_text'])
        my_results.append(
            {
                "point_index": i,
                "point": r[0]['summary_text'],
                "original_text": my_string,
                "sentiment": classify_point_sentiment(r[0]['summary_text']),
                "cat": cat,
                "sub_cat": sub_cat
            })
    return my_results


In [ ]:
new_df = pd.DataFrame()
for index, row in df.iterrows():
    temp_df = pd.DataFrame(get_points(row['Text']))
    temp_df['text_index'] = index
    new_df = new_df.append(temp_df, ignore_index=True)


In [13]:
new_df

,point_index,point,original_text,sentiment,cat,sub_cat,text_index
0,0,I have never owned a pair of apple airpods bef...,I have never owned a pair of apple airpods bef...,neutral,Quality,Performance,0
1,1,The airpods seem to pass the sniff test on my ...,the apple support site it did identify as 2nd ...,positive,Quality,Performance,0
2,2,When I connect the airpods to my iphone it doe...,it should say assembled in China for some and ...,negative,Quality,Performance,0
3,3,The iPhone 7 and 7 Plus have a 99.9% battery l...,battery life the moment I take them off the ch...,positive,Quality,Performance,0
4,4,When cranking the volume all the way and liste...,just nitpicking the sound on music is pretty g...,negative,Quality,Performance,0
5,5,These are real 2nd gen airpods but my skeptica...,to check out and these are real 2nd gen airpod...,negative,Quality,Performance,0
6,0,I have never owned un par of apple airpods bef...,I have never owned un par of apple airpods bef...,neutral,Quality,Performance,1
7,1,The airpods seem to pass the sniff test en my ...,the apple support site It did identify as 2nd ...,positive,Quality,Performance,1
8,2,When I connect the airpods to my iphone it doe...,"that it should say ""Assembled en China"" for so...",positive,Quality,Performance,1
9,3,The moment I take them off the charger and beg...,fast battery vida The moment I take them off t...,negative,Quality,Performance,1


In [14]:
new_df.to_csv('test_example.csv', index=True)